# Google Speech Commands Benchmark Tutorial

This tutorial aims to provide an insight on how the NeuroBench framework is organized and how you can use it to benchmark your own models!

## About Google Speech Commands:
Google Speech Commands is a keyword spotting task. Voice commands represent a natural and easily accessible modality for human-machine interaction. Keyword detection, in particular, is frequently employed in edge devices that operate in always-listening, wake-up situations, where it triggers more computationally demanding processes such as automatic speech recognition. Keyword spotting finds application in activating voice assistants, speech data mining, audio indexing, and phone call routing. Given that it generally operates in always-on and battery-powered edge scenarios, keyword detection represents a pertinent benchmark for energy-efficient neuromorphic solutions.
### Dataset:
The Google Speech Commands dataset (V2) is a commonly used dataset in assessing the performance of keyword spotting algorithms. The dataset consists of 105,829 1 second utterances of 35 different words from 2,618 distinct speakers. The data is encoded as linear 16-bit, single-channel, pulse code modulated values, at a 16 kHz sampling frequency.

### Benchmark Task:
The goal is to develop a model that trains using the designated train and validation sets, followed by an evaluation of generalization to a spearate test set. The task is a classification task.


First we will import the relevant libraries. These include the datasets, preprocessors and accumulators. To ensure your model to be compatible with the NeuroBench framework, we will import the wrapper for snnTorch models. This wrapper will not change your model. Finally, we import the Benchmark class, which will run the benchmark and calculate your metrics.

In [1]:
import torch
# import the dataloader
from torch.utils.data import DataLoader

# import the dataset, preprocessors and accumulators you want to use
from neurobench.datasets import SpeechCommands
from neurobench.preprocessing import S2SProcessor
from neurobench.accumulators import choose_max_count

# import the NeuroBench wrapper to wrap your snnTorch model for usage in the NeuroBench framework
from neurobench.models import SNNTorchModel
# import the benchmark class
from neurobench.benchmarks import Benchmark



For this tutorial, we will make use of the example architecture that is included in the NeuroBench framework.

In [2]:
# this is the network we will be using in this tutorial
from neurobench.examples.gsc.SNN import net

To get started, we will load our desired dataset in a dataloader:

In [3]:
test_set = SpeechCommands(path="data/speech_commands/", subset="testing")

test_set_loader = DataLoader(test_set, batch_size=500, shuffle=True)

FileNotFoundError: [Errno 2] No such file or directory: '/home/korneel/NeuroBench/algorithms_benchmarks/neurobench/examples/gsc/data/speech_commands/tmpy8couawl'

Next, load our model and wrap it in the corresponding NeuroBench wrapper. At the time of writing this tutorial, (V1.0) snnTorch is the only supported framework, therefore, we will wrap our snnTorch model in the SNNTorchModel() wrapper.

In [ ]:
net.load_state_dict(torch.load("neurobench/examples/gsc/model_data/s2s_gsc_snntorch", map_location=torch.device('cpu')))

# Wrap our net in the SNNTorchModel wrapper
model = SNNTorchModel(net)

Specify the preprocessor and postprocessor want to use. These will be applied to your data before feeding into the model, and to the output spikes respectively.
Available preprocessors and postprocessors can be found in neurobench/preprocessors and neurobench/accumulators respectively.

In [ ]:
preprocessors = [S2SProcessor()]
postprocessors = [choose_max_count]

Next specify the metrics which you want to calculate. The available metrics (V1.0 release) are:

static_metrics:
*   model_size
*   connection_sparsity
*   frequency


workload_metrics
*   activation_sparsity
*   multiply_accumulates
*   classification_accuracy

More accuracy metrics are available, for which the user is recommended to consult the documentation

More explanation on the metrics can be found on https://neurobench.ai/.

In [ ]:
static_metrics = ["footprint"]
workload_metrics = ["classification_accuracy"]

Next, we instanciate the benchmark. We have to specify the model, the dataloader, the preprocessors, the postprocessor and the list of the static and data metrics which we want to measure:

In [ ]:
benchmark = Benchmark(model, test_set_loader, preprocessors, postprocessors, [static_metrics, workload_metrics])

Now, let's run the benchmark and print our results!

In [ ]:
results = benchmark.run()
print(results)